In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'E:\\Document\\HK2-2022-2023\\DIPR\\Project\\research\\GrapeDiseaseDataset\\OriginalData\\train'
test_dir = 'E:\\Document\\HK2-2022-2023\\DIPR\\Project\\research\\GrapeDiseaseDataset\\OriginalData\\test'



In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

Found 7222 images belonging to 4 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

Found 1805 images belonging to 4 classes.


# First test model

In [ ]:
model = None
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


# Second test model with dropout

In [ ]:
model = None
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


# ResNet 50

In [ ]:
# Define the input shape
input_shape = (256, 256, 3)

# Define the number of classes
num_classes = 4

# Define the model architecture
inputs = tf.keras.Input(shape=input_shape)

# Stage 1
x = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2), padding='same')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

# Stage 2
shortcut = x
x = tf.keras.layers.Conv2D(64, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

# Stage 3
shortcut = x
x = tf.keras.layers.Conv2D(128, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 4
shortcut = x
x = tf.keras.layers.Conv2D(256, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 5
shortcut = x
x = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Final layers
x = tf.keras.layers.AveragePooling2D((7, 7))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

#Create Model
model = None
model = tf.keras.models.Model(inputs=inputs, outputs=x)
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


# With dropout

In [6]:
# Define the input shape
input_shape = (256, 256, 3)

# Define the number of classes
num_classes = 4

# Define the model architecture
inputs = tf.keras.Input(shape=input_shape)

# Stage 1
x = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2), padding='same')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

# Stage 2
shortcut = x
x = tf.keras.layers.Conv2D(64, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(64, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(256, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

# Stage 3
shortcut = x
x = tf.keras.layers.Conv2D(128, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(512, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 4
shortcut = x
x = tf.keras.layers.Conv2D(256, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(1024, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

#Stage 5
shortcut = x
x = tf.keras.layers.Conv2D(512, (1,1), strides=(2,2), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(512, (3,3), strides=(1,1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(2,2), padding='valid')(shortcut)
if x.shape != shortcut.shape:
    shortcut = tf.keras.layers.Conv2D(2048, (1,1), strides=(1,1), padding='valid')(shortcut)
x = tf.keras.layers.Add()([x, shortcut])
x = tf.keras.layers.Activation('relu')(x)

# Final layers
x = tf.keras.layers.AveragePooling2D((7, 7))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(4, activation='softmax')(x)

#Create Model
model = None
model = tf.keras.models.Model(inputs=inputs, outputs=x)
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [7]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator
)


Epoch 1/50


c:\Users\TVD\anaconda3\envs\tf\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


226/226 [==============================] - 144s 615ms/step - loss: 0.5014 - accuracy: 0.8272 - val_loss: 11.1346 - val_accuracy: 0.3025
Epoch 2/50
226/226 [==============================] - 88s 389ms/step - loss: 0.2400 - accuracy: 0.9133 - val_loss: 5.9220 - val_accuracy: 0.5169
Epoch 3/50
226/226 [==============================] - 89s 392ms/step - loss: 0.1638 - accuracy: 0.9420 - val_loss: 3.4079 - val_accuracy: 0.3524
Epoch 4/50
226/226 [==============================] - 90s 399ms/step - loss: 0.1438 - accuracy: 0.9495 - val_loss: 31.2247 - val_accuracy: 0.3745
Epoch 5/50
226/226 [==============================] - 89s 393ms/step - loss: 0.1175 - accuracy: 0.9597 - val_loss: 6.4138 - val_accuracy: 0.6305
Epoch 6/50
226/226 [==============================] - 91s 404ms/step - loss: 0.1072 - accuracy: 0.9636 - val_loss: 2.5214 - val_accuracy: 0.6510
Epoch 7/50
226/226 [==============================] - 91s 400ms/step - loss: 0.0668 - accuracy: 0.9785 - val_loss: 0.9579 - val_accuracy: 

In [8]:
model.evaluate(test_generator)

57/57 [==============================] - 5s 79ms/step - loss: 0.0690 - accuracy: 0.9789


[0.06896321475505829, 0.9789473414421082]

# Save first model

In [ ]:
# Save the trained model as a .h5 file
model.save('my_trained_model.h5')

# Save second model

In [ ]:
# Save the trained model as a .h5 file
model.save('my_trained_model2.h5')

# Save ResNet

In [ ]:
# Save the trained model as a .h5 file
model.save('my_trained_model3.h5')

# Save ResNet with dropout

In [ ]:
# Save the trained model as a .h5 file
model.save('my_trained_model4.h5')

# Save ResNet with dropout 30%

In [9]:
model.save('my_trained_model5.h5')

# GUI

In [10]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import ImageTk, Image
import numpy as np

# variable
file_path = None
# Load the saved model
model = tf.keras.models.load_model('my_trained_model5.h5')
img_height = img_width = 256
# Define the class names
class_names = ['Black Rot', 'ESCA', 'Healthy', 'Leaf Blight', ...]

# Create a function to preprocess the image
def preprocess_image(image):
    # Resize the image to match the input size of the model
    image = image.resize((img_height, img_width))
    # Convert the image to a numpy array
    image_array = np.array(image)
    # Normalize the pixel values to be between 0 and 1
    image_array = image_array / 255.0
    # Expand the dimensions of the image to match the input shape of the model
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

# Create a function to classify the image
def classify_image():
    global file_path
    if file_path == None:
        # Show an error message if there was a problem loading the image
        messagebox.showerror('Error', 'Please upload image!')
    # Load the image
    image = Image.open(file_path)
    # Preprocess the image
    image_array = preprocess_image(image)
    # Use the model to make predictions
    predictions = model.predict(image_array)
    model.evaluate(image_array)
    # # Get the class with the highest probability
    # predicted_class = np.argmax(predictions[0])
    # # Get the class name
    # class_name = class_names[predicted_class]
    # # Get the probability of the predicted class
    # class_prob = np.max(predictions[0])
    # Show a message box with the predicted class and percentage
    canvasInfor.delete('all')
    text = canvasInfor.create_text(90, 80, anchor=tk.CENTER, text = "Predict information \n Black Rot: {0}% \n ESCA: {1}% \n Healthy: {2}% \n Leaf Blight: {3}%".format(round( predictions[0][0]*100, 2),round( predictions[0][1]*100, 2),round( predictions[0][2]*100, 2),round( predictions[0][3]*100, 2)), font=("Arial", 14))
    # messagebox.showinfo('Classification Result', f'The image is classified as {class_name} with {class_prob} probability.')

# Create a function to handle the upload button click
def handle_upload():
    # Open a file dialog to select the image file
    global file_path
    file_path = filedialog.askopenfilename()
    # Check if a file was selected
    if file_path:
        try:
            # Load the image and display it in the GUI
            image = Image.open(file_path)
            image = image.resize((400, 400))
            image_tk = ImageTk.PhotoImage(image)
            canvas.itemconfig(canvas_image, image=image_tk)
            canvas.image = image_tk
        except:
            # Show an error message if there was a problem loading the image
            messagebox.showerror('Error', 'Could not open the image file.')

# Create the GUI window
window = tk.Tk()
window.title('Image Classifier')
window.geometry("900x650")

# Create a canvas to display banner
canvasBanner = tk.Canvas(window, width=900, height=180)
canvasBanner.place(x=0, y = 0)
canvas_imageBanner = canvasBanner.create_image(0, 0, anchor=tk.NW)
imageBanner = Image.open("./banner.png")
imageBanner = imageBanner.resize((900, 180))
image_tk = ImageTk.PhotoImage(imageBanner)
canvasBanner.itemconfig(canvas_imageBanner, image=image_tk)
canvasBanner.image = image_tk

# Create a canvas to display the image
canvas = tk.Canvas(window, width=400, height=400, bd=2, relief='solid')
canvas.place(x=50, y = 100 + 100)
canvas_image = canvas.create_image(0, 0, anchor=tk.NW)

# Create a canvas to display infor
canvasInfor = tk.Canvas(window, width=200, height=400, bd=1, relief='solid')
canvasInfor.place(x=500, y = 100 + 100)

# Create a button to upload the image
upload_button = tk.Button(window, text='Upload Image', command=handle_upload, width=12)
upload_button.place(x=760, y = 100 + 100)

classify_button = tk.Button(window, text='Classify', command=classify_image, width=12)
classify_button.place(x=760, y = 200 + 100)

# create a label
label = tk.Label(window, text="Select an model:")
label.place(x=760, y = 300 + 100)

# create a variable to hold the selected option
selected_option = tk.StringVar(window)

# set the default value of the variable
selected_option.set("ResNet 50 with dropout")

# create a selection box (option menu)
options = ["ResNet 50 with dropout", "ResNet 50", "Basic CNN", "Basic CNN with dropout", "ResNet 50 - imagenet"]
option_menu = tk.OptionMenu(window, selected_option, *options)
option_menu.config(width=23)
option_menu.place(x=710, y = 350 + 100)
def on_option_changed(*args):
    global model
    # get the new selected option
    new_option = selected_option.get()
    # do something with the new selected option
    print("Selected option:", new_option)
    if new_option == "ResNet 50 with dropout":
        model = None
        model = tf.keras.models.load_model('my_trained_model5.h5')
    elif new_option == "ResNet 50":
        model = None
        model = tf.keras.models.load_model('my_trained_model3.h5')
    elif new_option == "Basic CNN":
        model = None
        model = tf.keras.models.load_model('my_trained_model.h5')
    elif new_option == "ResNet 50 - imagenet":
        model = None
        model = tf.keras.models.load_model('grape_disease_model_1.h5')
    else:
        model = None
        model = tf.keras.models.load_model('my_trained_model2.h5')

# attach a trace to the selected_option variable
selected_option.trace("w", on_option_changed)
# Start the GUI main loop
window.mainloop()


1/1 [==============================] - 0s 21ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
